# Saving And Restoring TF Variables

TensorFlow variables are parameters of the computational graph. For most of Machine Learning problems these parameters are trained. Most of the times for Machine Learning problems the training of the variables might take hours even days.
But once you close the TensorFlow session you loose all the trained parameters. If you were to reuse the model, you have to train all over again.

TensorFlow provided the ability to save progress using __tf.train.Saver__.  

Lets see this in action.

In [32]:
import tensorflow as tf

# Saving Variables

In [33]:
#Lets declare few variables
save_file = './model.ckpt'
W = tf.Variable(tf.truncated_normal([2, 3]))
b = tf.Variable(tf.truncated_normal([3]))

saver = tf.train.Saver()
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    print(sess.run([W, b]))
    writer = tf.summary.FileWriter('./graphs', sess.graph)
    saver.save(sess, save_file)

writer.close()

[array([[ 0.77289367,  0.37127694,  0.28093734],
       [ 1.92050874, -0.69945496, -0.80512178]], dtype=float32), array([-1.5962317 , -0.92715842, -1.78852868], dtype=float32)]


# Loading Variables

In [34]:
tf.reset_default_graph()

save_file = './model.ckpt'
W = tf.Variable(tf.truncated_normal([2, 3]))
b = tf.Variable(tf.truncated_normal([3]))

saver = tf.train.Saver()

with tf.Session() as sess:
    saver.restore(sess, save_file)
    writer = tf.summary.FileWriter('./graphs', sess.graph)
    print(sess.run([W, b]))

writer.close()

INFO:tensorflow:Restoring parameters from ./model.ckpt
[array([[ 0.77289367,  0.37127694,  0.28093734],
       [ 1.92050874, -0.69945496, -0.80512178]], dtype=float32), array([-1.5962317 , -0.92715842, -1.78852868], dtype=float32)]


As you can see for restoring the variables we need to declare the corresponding tensors into which saved variables will be restored.

We dont need to run __tf.global_variables_initializer()__ when restoring the variables.


But there is a problem with approach above.

When we create tensors without giving them name TensorFlow automatically names the tensor.  
The format of the name will be <Type> for first and then <Type>_<Number> for rest of tensor.  
For __W__ variable the name will be __Variable__.    
For __b__ variable the name will be __Variable_1__.  


When we save the sess the checkpoint file will store the varibale value against the variable name.  
__Variable__ will have the value of variable __W__.  
__Variable_1__ will have the value of variable __b__.  

Let see what happens if we change order in which we declared variables while loading.  


In [35]:
tf.reset_default_graph()

save_file = './model.ckpt'
b = tf.Variable(tf.truncated_normal([3]))
W = tf.Variable(tf.truncated_normal([2, 3]))

saver = tf.train.Saver()

with tf.Session() as sess:
    saver.restore(sess, save_file)
    writer = tf.summary.FileWriter('./graphs', sess.graph)
    print(sess.run([W, b]))
    
writer.close()

INFO:tensorflow:Restoring parameters from ./model.ckpt


InvalidArgumentError: Assign requires shapes of both tensors to match. lhs shape= [3] rhs shape= [2,3]
	 [[Node: save/Assign = Assign[T=DT_FLOAT, _class=["loc:@Variable"], use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/cpu:0"](Variable, save/RestoreV2)]]

Caused by op 'save/Assign', defined at:
  File "/Users/arthur/anaconda/envs/tf-learn/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/arthur/anaconda/envs/tf-learn/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/arthur/anaconda/envs/tf-learn/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/arthur/anaconda/envs/tf-learn/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/arthur/anaconda/envs/tf-learn/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    i

Why error is thrown??  

Now during restoring we declared __b__ first and then __W__.  
So __b__ got name of __Variable__ and __W__ got name of __Variable_1__.

In the stored file we store value of __W__ against name __Variable__.  
While loading TensorFlow will load the __Variable__ value in tensor __b__.   
It will throw error since value of saved __W__ cannot be restored in __b__ as tensor shapes are not compatible.  
To fix this we need to set the name property while creating the __tf.Variables__.  


# Proper Saving of Varibles with name

In [36]:
#Lets declare few variables
save_file = './model.ckpt'
W = tf.Variable(tf.truncated_normal([2, 3]), name="W")
b = tf.Variable(tf.truncated_normal([3]), name="b")

saver = tf.train.Saver()
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    print(sess.run([W, b]))
    writer = tf.summary.FileWriter('./graphs', sess.graph)
    saver.save(sess, save_file)
    
writer.close()

[array([[ 0.04411612,  0.0101524 , -0.7080518 ],
       [-0.25548437,  1.58541822,  1.52689433]], dtype=float32), array([-0.4138411 , -0.89754373,  0.04580444], dtype=float32)]


# Proper Loading of Variables with name

In [37]:
tf.reset_default_graph()

save_file = './model.ckpt'
b = tf.Variable(tf.truncated_normal([3]), name="b")
W = tf.Variable(tf.truncated_normal([2, 3]), name="W")

saver = tf.train.Saver()

with tf.Session() as sess:
    saver.restore(sess, save_file)
    writer = tf.summary.FileWriter('./graphs', sess.graph)
    print(sess.run([W, b]))
    
writer.close()

INFO:tensorflow:Restoring parameters from ./model.ckpt
[array([[ 0.04411612,  0.0101524 , -0.7080518 ],
       [-0.25548437,  1.58541822,  1.52689433]], dtype=float32), array([-0.4138411 , -0.89754373,  0.04580444], dtype=float32)]
